Ch2 p62

Apply eq. 2.42 to compute partials and solve this system by Newton's method:

1. xyz - x^2 + y^2 = 1.34
2. xy - z^2 = 0.09
3. e^x - e^y + z = 0.41

There should be a solution near (1,1,1)

------
I am also going to try to use the perturbation idea to compute an approximate derivate and find the solution
that way. I.e.:

dxf1 = [f1(x+dx,y,z) - f1(x,y,z)]/dx

dyf1 = [f1(x,y+dy,z) - f1(x,y,z)]/dy

dzf1 = [f1(x,y,z+dz) - f1(x,y,z)]/dz

...

dzf3 = [f1(x,y,z+dz) - f1(x,y,z)]/dz

In [49]:
import numpy as np
from naf.linalg_exp import gedo, dosv, set_options
import math

set_options(precision=5)

First implementation using eq. 2.42

In [52]:
def a_m(w):
    x = w[0]
    y = w[1]
    z = w[2]
    
    dxf1 = lambda x,y,z: y*z - 2*x
    dyf1 = lambda x,y,z: x*z + 2*y
    dzf1 = lambda x,y,z: x*y

    dxf2 = lambda x,y,z: y
    dyf2 = lambda x,y,z: x
    dzf2 = lambda x,y,z: -2.0*z

    dxf3 = lambda x,y,z: math.exp(x)
    dyf3 = lambda x,y,z: -1.0*math.exp(y)
    dzf3 = lambda x,y,z: 1.0

    return np.array([[dxf1(x,y,z), dyf1(x,y,z), dzf1(x,y,z)],
                     [dxf2(x,y,z), dyf2(x,y,z), dzf2(x,y,z)],
                     [dxf3(x,y,z), dyf3(x,y,z), dzf3(x,y,z)]])

def b_v(w):
    x = w[0]
    y = w[1]
    z = w[2]
    
    f1 = lambda x,y,z: x*y*z - x**2 + y**2 - 1.34
    f2 = lambda x,y,z: x*y - z**2 - 0.09
    f3 = lambda x,y,z: math.exp(x) - math.exp(y) + z - 0.41
    
    return np.array([f1(x,y,z), f2(x,y,z), f3(x,y,z)])


def three_eq_newton(x0):
    
    tol = 0.0001
    dx = np.array([tol*10, tol*10, tol*10])
    num_iter = 0
    max_iter = 20
    
    while (abs(dx[0])>tol or abs(dx[1])>tol or abs(dx[2])>tol) and num_iter < max_iter:
        
        a = a_m(x0)
        b = -1*b_v(x0) #the blasted -1, agh! gotta remember that
        
        lu,ov = gedo(a)
        #the [ov] at the end re-orders the result from dosv
        #to match the original order of equations so the right
        #difference gets added to x0
        dx = dosv(lu,ov,b)[ov]
        
        x0 = x0 + dx
        num_iter += 1
        
    return x0, dx, num_iter

w1 = np.array([1.0,1.0,1.0], dtype=float)
x1,dx1,ni1 = three_eq_newton(w1)
print(x1,dx1,ni1)

[0.90221 1.10034 0.95013] [-2.e-05 -1.e-05  0.e+00] 3


Second implementation using perturbations

In [51]:
f1 = lambda x,y,z: x*y*z - x**2 + y**2 - 1.34
f2 = lambda x,y,z: x*y - z**2 - 0.09
f3 = lambda x,y,z: math.exp(x) - math.exp(y) + z - 0.41

def a_m(w, d):
    x = w[0]
    y = w[1]
    z = w[2]
    
    dxf1 = f1(x+d, y, z)
    dyf1 = f1(x, y+d, z)
    dzf1 = f1(x, y, z+d)
    
    dxf2 = f2(x+d, y, z)
    dyf2 = f2(x,y+d,z)
    dzf2 = f2(x,y,z+d)
    
    dxf3 = f3(x+d,y,z)
    dyf3 = f3(x,y+d,z)
    dzf3 = f3(x,y,z+d)
    
    return np.array([[dxf1, dyf1, dzf1],
                     [dxf2, dyf2, dzf2],
                     [dxf3, dyf3, dzf3]])

def b_v(w):
    x = w[0]
    y = w[1]
    z = w[2]
    
    return np.array([f1(x,y,z), f2(x,y,z), f3(x,y,z)])

def three_eq_newton(x0, d,mi):
    
    tol = 0.0001
    dx = np.full(3,tol*10)
    num_iter = 0
    max_iter = mi
    
    while (abs(dx[0])>tol or abs(dx[1])>tol or abs(dx[2])>tol) and num_iter < max_iter:
        
        a = a_m(x0, d)
        b = -1*b_v(x0)
        
        lu,ov = gedo(a)
        #[ov] at end reorders to original equation order
        dx = dosv(lu,ov,b)[ov]

        x0 = x0 + dx
        num_iter += 1
        
    return x0, dx, num_iter

w2 = np.ones(3)
d = 2.5
mi = 50
x2, dx2, ni2 = three_eq_newton(w2, d, mi)
print(x2, dx2, ni2)

w2 = np.ones(3)
d = 1.0
mi = 20
x2, dx2, ni2 = three_eq_newton(w2, d,mi)
print(x2, dx2, ni2)

w2 = np.ones(3)
d = 0.5
mi = 1000
x2, dx2, ni2 = three_eq_newton(w2, d,mi)
print(x2, dx2, ni2)

[0.90306 1.10002 0.95075] [-9.e-05  4.e-05 -7.e-05] 48
[0.90224 1.10032 0.95015] [-3.e-05  4.e-05 -2.e-05] 9
[0.96047 1.14861 0.95846] [0.09579 0.07972 0.01377] 1000


The second approach does work, of course. However, I find it interesting how much the 'delta' value, d, effects the speed and even ability of convergence. It's like if you don't perturb the system enough it won't converge and if you preturb it to much it won't converge. 

The implementation here is simple in that it uses the same delta for each equation. I wonder, and would bet, that each equation has some optimal delta that would speed up the convergence but I'm not going to explore that here. 

I initially thought that since the definition of a derivative has delta -> 0 that a small value of delta would offer the best solution but it does not. I think because theoretically that definition makes sense for the slope at point. However, for application the slope must be taken over some finitie distance and too small of a distance the computation will become inaccurate or unstable. The slope is kind like an average you need a reasonable amount of data to capture the behavior of a system. 

Ok, enough rambling. 